In [1]:
import pandas as pd
import numpy as np
import argparse
import time
import csv
import pickle
import operator
import datetime
import os
import math
import torch
from torch import nn
from torch.nn import Module, Parameter
import torch.nn.functional as F
from torch.nn import TransformerEncoder
from torch.nn import TransformerEncoderLayer

In [2]:
# initial preprocess so it can be used for sr-san
# we generate artificial dates based on sequence

data = open("dataset/computers_sample_train.txt", "r")

session_id = 0
seq = 1
data_array = []

def formated_seq(seq): 
    holder = 1
    if seq < 10:
        holder = "0" + str(seq)
    else:
        holder = str(seq)
    return holder

for line in data:
    for micro in line.split(" "): 
        if seq < 30:
            
            day = "+2001-01-" + formated_seq(seq)
        elif seq < 60:
             day = "+2001-02-0" + formated_seq(seq-29)
        else:
            day = "+2001-03-0" + str(np.random.randint(1,30))
        line = str(session_id) + "+NA+" + micro + day;
        line = line.strip("\n ")
        data_array.append(line)
        seq += 1
    seq = 1
    session_id += 1
    if session_id > 100: 
        break

In [3]:
# then we produce a data frame to use

In [4]:
data_pf = pd.DataFrame(data_array)
data_pf.columns = ["data"]
data_pf = data_pf.data.str.split(pat='+',expand=True)
data_pf.columns = ["session_id",  "user_id", "item_item", "behavior_type", "item_id", "timeframe", "dwell_time", "eventdate"]
data_pf

,session_id,user_id,item_item,behavior_type,item_id,timeframe,dwell_time,eventdate
0,0,NA,2885143,5,683,9,4,2001-01-01
1,0,NA,3282626,5,679,3,52,2001-01-02
2,0,NA,3282626,7,679,49,52,2001-01-03
3,0,NA,3203662,5,679,191,10,2001-01-04
4,0,NA,1836048,5,681,122,9,2001-01-05
...,...,...,...,...,...,...,...,...
2857,100,NA,2948633,6,688,139,137,2001-01-18
2858,100,NA,1250967,5,2694,21,21,2001-01-19
2859,100,NA,2131674,5,2694,51,51,2001-01-20
2860,100,NA,1892003,5,2694,20,20,2001-01-21


In [5]:
# delete unused columns
del(data_pf['behavior_type'])
del(data_pf['item_item'])
del(data_pf['dwell_time'])

In [6]:
# save it
from pathlib import Path
filepath = Path('jd_computers.csv', index=False)  
data_pf.to_csv(filepath) 
print("--> Saved data")

--> Saved data


In [7]:
####################################################################################
####################################################################################
####### SR-SAN CODE OBTAINED FROM https://github.com/GalaxyCruiser/SR-SAN ##########
####################################################################################
####################################################################################


# PREPROCESS SR-SAN

In [8]:
import argparse
import time
import csv
import pickle
import operator
import datetime
import os

dataset = 'jd_computers.csv'

print("-- Starting @ %ss" % datetime.datetime.now())
with open(dataset, "r") as f:
    reader = csv.DictReader(f, delimiter=',')
    sess_clicks = {}
    sess_date = {}
    ctr = 0
    curid = -1
    curdate = None
    for data in reader:
        sessid = data['session_id']
        if curdate and not curid == sessid:
            date = ''
            try:
                date = time.mktime(time.strptime(curdate, '%Y-%m-%d'))
            except: 
                date = time.mktime(time.strptime("2021-01-01", '%Y-%m-%d'))
                
            sess_date[curid] = date
        curid = sessid
        try:
            item = data['item_id'], int(data['timeframe'])
        except: 
            item = data['item_id'], int(1)
        curdate = ''
        curdate = data['eventdate']

        if sessid in sess_clicks:
            sess_clicks[sessid] += [item]
        else:
            sess_clicks[sessid] = [item]
        ctr += 1
    date = ''
    try:
        date = time.mktime(time.strptime(curdate, '%Y-%m-%d'))
    except: 
        date = time.mktime(time.strptime("2021-01-01", '%Y-%m-%d'))
        for i in list(sess_clicks):
            sorted_clicks = sorted(sess_clicks[i], key=operator.itemgetter(1))
            sess_clicks[i] = [c[0] for c in sorted_clicks]
    sess_date[curid] = date
print("-- Reading data @ %ss" % datetime.datetime.now())

# Filter out length 1 sessions
for s in list(sess_clicks):
    if len(sess_clicks[s]) == 1:
        del sess_clicks[s]
        del sess_date[s]

# Count number of times each item appears
iid_counts = {}
for s in sess_clicks:
    seq = sess_clicks[s]
    for iid in seq:
        if iid in iid_counts:
            iid_counts[iid] += 1
        else:
            iid_counts[iid] = 1

sorted_counts = sorted(iid_counts.items(), key=operator.itemgetter(1))

length = len(sess_clicks)
for s in list(sess_clicks):
    curseq = sess_clicks[s]
    filseq = list(filter(lambda i: iid_counts[i] >= 5, curseq))
    if len(filseq) < 2:
        del sess_clicks[s]
        del sess_date[s]
    else:
        sess_clicks[s] = filseq

# Split out test set based on dates
dates = list(sess_date.items())
maxdate = dates[0][1]

for _, date in dates:
    if maxdate < date:
        maxdate = date

# 7 days for test
splitdate = 0
splitdate = maxdate - 86400 * 7

print('Splitting date', splitdate)      # Yoochoose: ('Split date', 1411930799.0)
tra_sess = filter(lambda x: x[1] < splitdate, dates)
tes_sess = filter(lambda x: x[1] > splitdate, dates)

# Sort sessions by date
tra_sess = sorted(tra_sess, key=operator.itemgetter(1))     # [(session_id, timestamp), (), ]
tes_sess = sorted(tes_sess, key=operator.itemgetter(1))     # [(session_id, timestamp), (), ]
print(len(tra_sess))    # 186670    # 7966257
print(len(tes_sess))    # 15979     # 15324
print(tra_sess[:3])
print(tes_sess[:3])
print("-- Splitting train set and test set @ %ss" % datetime.datetime.now())

# Choosing item count >=5 gives approximately the same number of items as reported in paper
item_dict = {}
# Convert training sessions to sequences and renumber items to start from 1
def obtian_tra():
    train_ids = []
    train_seqs = []
    train_dates = []
    item_ctr = 1
    for s, date in tra_sess:
        seq = sess_clicks[s]
        outseq = []
        for i in seq:
            if i in item_dict:
                outseq += [item_dict[i]]
            else:
                outseq += [item_ctr]
                item_dict[i] = item_ctr
                item_ctr += 1
        if len(outseq) < 2:  # Doesn't occur
            continue
        train_ids += [s]
        train_dates += [date]
        train_seqs += [outseq]
    print(item_ctr)     # 43098, 37484
    return train_ids, train_dates, train_seqs


# Convert test sessions to sequences, ignoring items that do not appear in training set
def obtian_tes():
    test_ids = []
    test_seqs = []
    test_dates = []
    for s, date in tes_sess:
        seq = sess_clicks[s]
        outseq = []
        for i in seq:
            if i in item_dict:
                outseq += [item_dict[i]]
        if len(outseq) < 2:
            continue
        test_ids += [s]
        test_dates += [date]
        test_seqs += [outseq]
    return test_ids, test_dates, test_seqs


tra_ids, tra_dates, tra_seqs = obtian_tra()
tes_ids, tes_dates, tes_seqs = obtian_tes()


def process_seqs(iseqs, idates):
    out_seqs = []
    out_dates = []
    labs = []
    ids = []
    for id, seq, date in zip(range(len(iseqs)), iseqs, idates):
        for i in range(1, len(seq)):
            tar = seq[-i]
            labs += [tar]
            out_seqs += [seq[:-i]]
            out_dates += [date]
            ids += [id]
    return out_seqs, out_dates, labs, ids


tr_seqs, tr_dates, tr_labs, tr_ids = process_seqs(tra_seqs, tra_dates)
te_seqs, te_dates, te_labs, te_ids = process_seqs(tes_seqs, tes_dates)
tra = (tr_seqs, tr_labs)
tes = (te_seqs, te_labs)
print(len(tr_seqs))
print(len(te_seqs))
print(tr_seqs[:3], tr_dates[:3], tr_labs[:3])
print(te_seqs[:3], te_dates[:3], te_labs[:3])
all = 0

for seq in tra_seqs:
    all += len(seq)
for seq in tes_seqs:
    all += len(seq)
print('avg length: ', all/(len(tra_seqs) + len(tes_seqs) * 1.0))

if not os.path.exists('jd'):
    os.makedirs('jd')
pickle.dump(tra, open('jd/train.txt', 'wb'))
pickle.dump(tes, open('jd/test.txt', 'wb'))
pickle.dump(tra_seqs, open('jd/all_train_seq.txt', 'wb'))

print('Done.')

-- Starting @ 2022-06-17 19:45:36.259948s
-- Reading data @ 2022-06-17 19:45:36.275980s
Splitting date 1608822000.0
39
37
[('7', 979657200.0), ('68', 979657200.0), ('55', 979743600.0)]
[('0', 1609426800.0), ('1', 1609426800.0), ('3', 1609426800.0)]
-- Splitting train set and test set @ 2022-06-17 19:45:36.277978s
84
299
225
[[1, 2, 3, 4, 5, 6, 7, 8, 8, 3], [1, 2, 3, 4, 5, 6, 7, 8, 8], [1, 2, 3, 4, 5, 6, 7, 8]] [979657200.0, 979657200.0, 979657200.0] [9, 3, 8]
[[57, 77, 77, 77], [57, 77, 77], [57, 77]] [1609426800.0, 1609426800.0, 1609426800.0] [77, 77, 77]
avg length:  8.594202898550725
Done.


In [9]:
# SR-SAN MAIN CODE
# above we can see the amount of nodes already, but we can make sure
data = open("jd/train.txt", "r")
obj = pd.read_pickle('jd/train.txt')
df = pd.DataFrame(obj)
nodes = df.max(axis=1)[1] + 1 
print("Nodes to use = " + str(nodes))

Nodes to use = 84


In [10]:
# utils


def data_masks(all_usr_pois, item_tail):
    us_lens = [len(upois) for upois in all_usr_pois]
    len_max = max(us_lens)
    us_pois = [upois + item_tail * (len_max - le) for upois, le in zip(all_usr_pois, us_lens)]
    us_msks = [[1] * le + [0] * (len_max - le) for le in us_lens]
    return us_pois, us_msks, len_max


def split_validation(train_set, valid_portion):
    train_set_x, train_set_y = train_set
    n_samples = len(train_set_x)
    sidx = np.arange(n_samples, dtype='int32')
    np.random.shuffle(sidx)
    n_train = int(np.round(n_samples * (1. - valid_portion)))
    valid_set_x = [train_set_x[s] for s in sidx[n_train:]]
    valid_set_y = [train_set_y[s] for s in sidx[n_train:]]
    train_set_x = [train_set_x[s] for s in sidx[:n_train]]
    train_set_y = [train_set_y[s] for s in sidx[:n_train]]

    return (train_set_x, train_set_y), (valid_set_x, valid_set_y)


class Data():
    def __init__(self, data, shuffle=False, graph=None):
        inputs = data[0]
        inputs, mask, len_max = data_masks(inputs, [0])
        self.inputs = np.asarray(inputs)
        self.mask = np.asarray(mask)
        self.len_max = len_max
        self.targets = np.asarray(data[1])
        self.length = len(inputs)
        self.shuffle = shuffle
        self.graph = graph

    def generate_batch(self, batch_size):
        if self.shuffle:
            shuffled_arg = np.arange(self.length)
            np.random.shuffle(shuffled_arg)
            self.inputs = self.inputs[shuffled_arg]
            self.mask = self.mask[shuffled_arg]
            self.targets = self.targets[shuffled_arg]
        n_batch = int(self.length / batch_size)
        if self.length % batch_size != 0:
            n_batch += 1
        slices = np.split(np.arange(n_batch * batch_size), n_batch)
        slices[-1] = slices[-1][:(self.length - batch_size * (n_batch - 1))]
        return slices

    def get_slice(self, i):
        inputs, mask, targets = self.inputs[i], self.mask[i], self.targets[i]
        items, n_node, A, alias_inputs = [], [], [], []
        for u_input in inputs:
            n_node.append(len(np.unique(u_input)))
        max_n_node = np.max(n_node)
        for u_input in inputs:
            node = np.unique(u_input)
            items.append(node.tolist() + (max_n_node - len(node)) * [0])
            u_A = np.zeros((max_n_node, max_n_node))
            for i in np.arange(len(u_input) - 1):
                if u_input[i + 1] == 0:
                    break
                u = np.where(node == u_input[i])[0][0]
                v = np.where(node == u_input[i + 1])[0][0]
                u_A[u][v] = 1
            u_sum_in = np.sum(u_A, 0)
            u_sum_in[np.where(u_sum_in == 0)] = 1
            u_A_in = np.divide(u_A, u_sum_in)
            u_sum_out = np.sum(u_A, 1)
            u_sum_out[np.where(u_sum_out == 0)] = 1
            u_A_out = np.divide(u_A.transpose(), u_sum_out)
            u_A = np.concatenate([u_A_in, u_A_out]).transpose()
            A.append(u_A)
            alias_inputs.append([np.where(node == i)[0][0] for i in u_input])
        return alias_inputs, A, items, mask, targets

In [11]:
# main model
class SelfAttentionNetwork(Module):
    def __init__(self, opt, n_node):
        super(SelfAttentionNetwork, self).__init__()
        self.hidden_size = opt.hiddenSize
        self.n_node = n_node
        self.batch_size = opt.batchSize
        self.embedding = nn.Embedding(self.n_node, self.hidden_size)
        self.transformerEncoderLayer = TransformerEncoderLayer(d_model=self.hidden_size, nhead=opt.nhead,dim_feedforward=self.hidden_size * opt.feedforward)
        self.transformerEncoder = TransformerEncoder(self.transformerEncoderLayer, opt.layer)
        self.loss_function = nn.CrossEntropyLoss()
        self.optimizer = torch.optim.Adam(self.parameters(), lr=opt.lr, weight_decay=opt.l2)
        self.scheduler = torch.optim.lr_scheduler.StepLR(self.optimizer, step_size=opt.lr_dc_step, gamma=opt.lr_dc)
        self.reset_parameters()

    def reset_parameters(self):
        stdv = 1.0 / math.sqrt(self.hidden_size)
        for weight in self.parameters():
            weight.data.uniform_(-stdv, stdv)

    def compute_scores(self, hidden, mask):
        ht = hidden[torch.arange(mask.shape[0]).long(), torch.sum(mask, 1) - 1]  # batch_size x latent_size
        b = self.embedding.weight[1:]  # n_nodes x latent_size
        scores = torch.matmul(ht, b.transpose(1, 0))
        return scores

    def forward(self, inputs, A):
        hidden = self.embedding(inputs)
        hidden = hidden.transpose(0,1).contiguous()
        hidden = self.transformerEncoder(hidden)
        hidden = hidden.transpose(0,1).contiguous()
        return hidden


def trans_to_cuda(variable):
    if torch.cuda.is_available():
        return variable.cuda()
    else:
        return variable


def trans_to_cpu(variable):
    if torch.cuda.is_available():
        return variable.cpu()
    else:
        return variable


def forward(model, i, data):
    alias_inputs, A, items, mask, targets = data.get_slice(i)
    alias_inputs = trans_to_cuda(torch.Tensor(alias_inputs).long())
    items = trans_to_cuda(torch.Tensor(items).long())
    A = trans_to_cuda(torch.Tensor(A).float())
    mask = trans_to_cuda(torch.Tensor(mask).long())
    hidden = model(items, A)
    get = lambda i: hidden[i][alias_inputs[i]]
    seq_hidden = torch.stack([get(i) for i in torch.arange(len(alias_inputs)).long()])
    return targets, model.compute_scores(seq_hidden, mask)


def train_test(model, train_data, test_data):    
    print('start training: ', datetime.datetime.now())
    model.train()
    total_loss = 0.0
    slices = train_data.generate_batch(model.batch_size)
    for i, j in zip(slices, np.arange(len(slices))):
        model.optimizer.zero_grad()
        targets, scores = forward(model, i, train_data)
        targets = trans_to_cuda(torch.Tensor(targets).long())
        loss = model.loss_function(scores, targets - 1)
        loss.backward()
        model.optimizer.step()
        total_loss += loss
        if j % int(len(slices) / 5 + 1) == 0:
            print('[%d/%d] Loss: %.4f' % (j, len(slices), loss.item()))
    print('\tLoss:\t%.3f' % total_loss)

    print('start predicting: ', datetime.datetime.now())
    model.eval()
    hit, mrr = [], []
    slices = test_data.generate_batch(model.batch_size)
    for i in slices:
        targets, scores = forward(model, i, test_data)
        sub_scores = scores.topk(5)[1]
        sub_scores = trans_to_cpu(sub_scores).detach().numpy()
        for score, target, mask in zip(sub_scores, targets, test_data.mask):
            hit.append(np.isin(target - 1, score))
            if len(np.where(score == target - 1)[0]) == 0:
                mrr.append(0)
            else:
                mrr.append(1 / (np.where(score == target - 1)[0][0] + 1))
    hit = np.mean(hit) * 100
    mrr = np.mean(mrr) * 100
    model.scheduler.step()
    return hit, mrr

In [14]:
#opt["batchSize"] = 100 # input batch size
#opt["hiddenSize"] = 96 # hidden state size   
#opt["nhead"] = 2 # the number of heads of multi-head attention    
#opt["layer"] = 1 # number of SAN layers   
#opt["feedforward"] = 4 # the multiplier of hidden state size   
#opt["epoch"] = 12 # the number of epochs to train for
#opt["lr"] = 0.001 # learning rate
#opt["lr_dc"] = 0.1 # learning rate decay rate
#opt["lr_dc_step"] = 3 # the number of steps after which the learning rate decay
#opt["l2"] = 1e-5 # l2 penalty
#opt["patience"] = 10 # the number of epoch to wait before early stop
#opt["validation"] = "store_true" # validatiob
#opt["valid_portion"] = 0.1 #split the portion of training set as validation set

class Opt:
    dataset = 'jd'
    batchSize = 100
    hiddenSize = 96
    nhead = 2
    layer = 1
    feedforward = 4
    epoch = 12
    lr = 0.001
    lr_dc = 0.1
    lr_dc_step = 3
    l2 = 1e-5
    patience = 10
    validation = 'store_true'
    valid_portion = 0.1
    
opt = Opt()

In [15]:
def main():
    train_data = pickle.load(open(opt.dataset + '/train.txt', 'rb'))
    if opt.validation:
        train_data, valid_data = split_validation(train_data, opt.valid_portion)
        test_data = valid_data
    else:
        test_data = pickle.load(open(opt.dataset + '/test.txt', 'rb'))

    train_data = Data(train_data, shuffle=True)
    test_data = Data(test_data, shuffle=False)
    n_node = 85


    model = trans_to_cuda(SelfAttentionNetwork(opt, n_node))

    start = time.time()
    best_result = [0, 0]
    best_epoch = [0, 0]
    bad_counter = 0
    for epoch in range(opt.epoch):
        print('-------------------------------------------------------')
        print('epoch: ', epoch)
        hit, mrr = train_test(model, train_data, test_data)
        flag = 0
        if hit >= best_result[0]:
            best_result[0] = hit
            best_epoch[0] = epoch
            flag = 1
        if mrr >= best_result[1]:
            best_result[1] = mrr
            best_epoch[1] = epoch
            flag = 1
        print('Best Result:')
        print('\tRecall@20:\t%.4f\tMMR@20:\t%.4f\tEpoch:\t%d,\t%d'% (best_result[0], best_result[1], best_epoch[0], best_epoch[1]))
        bad_counter += 1 - flag
        if bad_counter >= opt.patience:
            break
    print('-------------------------------------------------------')
    end = time.time()
    print("Run time: %f s" % (end - start))


In [16]:
main()

-------------------------------------------------------
epoch:  0
start training:  2022-06-17 19:46:00.091034
[0/3] Loss: 4.4471
[1/3] Loss: 4.4324
[2/3] Loss: 4.4272
	Loss:	13.307
start predicting:  2022-06-17 19:46:00.226034
Best Result:
	Recall@20:	6.6667	MMR@20:	1.7778	Epoch:	0,	0
-------------------------------------------------------
epoch:  1
start training:  2022-06-17 19:46:00.236033
[0/3] Loss: 4.4101


C:\Users\ferimaroo\AppData\Local\Temp\ipykernel_17576\834171849.py:53: UserWarning: Creating a tensor from a list of numpy.ndarrays is extremely slow. Please consider converting the list to a single numpy.ndarray with numpy.array() before converting to a tensor. (Triggered internally at  ..\torch\csrc\utils\tensor_new.cpp:210.)
  A = trans_to_cuda(torch.Tensor(A).float())


[1/3] Loss: 4.4237
[2/3] Loss: 4.4135
	Loss:	13.247
start predicting:  2022-06-17 19:46:00.346035
Best Result:
	Recall@20:	10.0000	MMR@20:	2.7778	Epoch:	1,	1
-------------------------------------------------------
epoch:  2
start training:  2022-06-17 19:46:00.356035
[0/3] Loss: 4.3979
[1/3] Loss: 4.4007
[2/3] Loss: 4.3966
	Loss:	13.195
start predicting:  2022-06-17 19:46:00.470033
Best Result:
	Recall@20:	10.0000	MMR@20:	3.6111	Epoch:	2,	2
-------------------------------------------------------
epoch:  3
start training:  2022-06-17 19:46:00.480064
[0/3] Loss: 4.3956
[1/3] Loss: 4.3826
[2/3] Loss: 4.3812
	Loss:	13.159
start predicting:  2022-06-17 19:46:00.598033
Best Result:
	Recall@20:	10.0000	MMR@20:	3.6111	Epoch:	3,	3
-------------------------------------------------------
epoch:  4
start training:  2022-06-17 19:46:00.608035
[0/3] Loss: 4.3852
[1/3] Loss: 4.3824
[2/3] Loss: 4.3904
	Loss:	13.158
start predicting:  2022-06-17 19:46:00.719033
Best Result:
	Recall@20:	13.3333	MMR@20:	